In [ ]:
!pip install labelme


  Using cached Pillow-10.0.1-cp310-cp310-manylinux_2_28_x86_64.whl (3.6 MB)
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0


In [1]:
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt  # Install dependencies


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [11]:
!pip install -q ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.1 MB/s eta 0:00:00


In [23]:
!mkdir -p /content/dataset/images
!mkdir -p /content/dataset/labels
!cp /content/your_images/*.jpg /content/dataset/images/
!cp /content/yolo_annotations/*.txt /content/dataset/labels/

In [14]:
%cd /content/yolov5
!pip install comet_ml
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 100 --data /content/yolov5/data.yml --weights /content/yolov5/models/yolov5s.yaml --cache


/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.4/677.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 64.0 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0
2024-07-02 18:17:49.538702: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 18:17:49.538759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT

In [2]:
import json
import os
import cv2

def convert_labelme_to_yolo(json_dir, output_dir, img_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            with open(os.path.join(json_dir, json_file)) as f:
                data = json.load(f)

            image_path = os.path.join(img_dir, data['imagePath'])
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            yolo_annotations = []
            for shape in data['shapes']:
                label = shape['label']
                points = shape['points']
                x_min = min([p[0] for p in points])
                y_min = min([p[1] for p in points])
                x_max = max([p[0] for p in points])
                y_max = max([p[1] for p in points])

                x_center = (x_min + x_max) / 2 / width
                y_center = (y_min + y_max) / 2 / height
                bbox_width = (x_max - x_min) / width
                bbox_height = (y_max - y_min) / height
                label_id = label_mapping[label]
                yolo_annotations.append(f"{label_id} {x_center} {y_center} {bbox_width} {bbox_height}")
            txt_file = os.path.join(output_dir, os.path.splitext(json_file)[0] + '.txt')
            with open(txt_file, 'w') as f:
                f.write('\n'.join(yolo_annotations))
label_mapping = {
    'love': 0,
    'not': 1,
    'ok': 2,
    'victory': 3,
    'hi': 4,
    'four': 5
}

json_dir = '/content/your_labelme_jsons'
output_dir = '/content/yolo_annotations'
img_dir = '/content/your_images'

convert_labelme_to_yolo(json_dir, output_dir, img_dir)


In [15]:
from IPython.display import display, Javascript

def get_video():
    js = Javascript('''
    async function recordVideo() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      video.play();

      // Create media recorder
      const mediaRecorder = new MediaRecorder(stream);
      let chunks = [];
      mediaRecorder.ondataavailable = (e) => {
        chunks.push(e.data);
      };
      mediaRecorder.start();

      // Record video
      await new Promise((resolve) => setTimeout(resolve, 5000));
      mediaRecorder.stop();

      // Stop video and remove element
      stream.getTracks().forEach(track => track.stop());
      video.remove();
      div.remove();

      // Save video
      const blob = new Blob(chunks, {type: 'video/mp4'});
      const url = URL.createObjectURL(blob);
      const a = document.createElement('a');
      a.href = url;
      a.download = 'webcam.mp4';
      a.click();
    }
    recordVideo();
    ''')
    display(js)

get_video()


<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
uploaded = files.upload()


Saving data.yml to data.yml


In [24]:
import torch
import cv2
from google.colab.patches import cv2_imshow
# model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', source='local')
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Function to process video file
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        results.render()
        for *box, conf, cls in results.xyxy[0]:
            x1, y1, x2, y2 = map(int, box)
            label = model.names[int(cls)]
            confidence = conf.item()
            print(f"Label: {label}, Confidence: {confidence:.2f}, Bounding box: ({x1}, {y1}, {x2}, {y2})")
        frame = results.imgs[0]
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
process_video('/content/webcam.mp4')


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-2 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [22]:
!python /content/yolov5/detect.py --source /content/webcam.mp4

detect: weights=yolov5s.pt, source=/content/webcam.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-334-g100a423b Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 313, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 308, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/detect.py", line 129, in run
